<a href="https://colab.research.google.com/github/farhan885/Saylani-Python-Assignment/blob/master/A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# My first CoLab

In [13]:

from sklearn.externals import joblib
from sklearn import datasets
from skimage.feature import hog
from sklearn.svm import LinearSVC
import numpy as np
from collections import Counter

dataset = datasets.fetch_openml('mnist_784')

features = np.array(dataset.data, )
labels = np.array(dataset.target)

list_hog_fd = []

for feature in features:
    fd = hog(feature.reshape(28,28), orientations=9, pixels_per_cell=(14,14), cells_per_block=(1,1), visualize=False)
    list_hog_fd.append(fd)
    
hog_features = np.array(list_hog_fd, 'float64')
print('Count of digits in datasets : ', Counter(labels))

clf = LinearSVC()
clf.fit(hog_features,labels)

joblib.dump(clf, "digits_cls.pkl", compress=3)


Count of digits in datasets :  Counter({'1': 7877, '7': 7293, '3': 7141, '2': 6990, '9': 6958, '0': 6903, '6': 6876, '8': 6825, '4': 6824, '5': 6313})


['digits_cls.pkl']

In [40]:
import cv2
from sklearn.externals import joblib
from skimage.feature import hog
import numpy as np
from google.colab.patches import cv2_imshow


clf = joblib.load('digits_cls.pkl')
im = cv2.imread('pic1.jpg', cv2.IMREAD_UNCHANGED)
im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5,5),0)

ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)

ctrs,hier = cv2.findContours(im_th.copy(), cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)

rects = [cv2.boundingRect(ctr) for ctr in ctrs]

for rect in rects:
  cv2.rectangle(im,(rect[0],rect[1]), (rect[0]+rect[2], rect[1]+rect[3]), (0,255,0), 3 )

  leng = int(rect[3] * 1.6)
  pt1 = int(rect[1] + rect[3] // 2 - leng // 2)
  pt2 = int(rect[0] + rect[2] // 2 - leng // 2)
  roi = im_th[pt1:pt1+leng, pt2:pt2+leng]

  roi = cv2.resize(roi, (150,108), interpolation=cv2.INTER_AREA)
  roi = cv2.dilate(roi, (3,3))

  roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(960,1280), cells_per_block=(1,1), visualize=False)
  nbr = clf.predict(np.array([roi_hog_fd]))
  cv2.putText(im, str(int(nbr[0])), (rect[0], rect[1]), cv2.FONT_HERSHEY_COMPLEX, 2, (0,255,255), 3 )

cv2_imshow(im)
cv2.waitKey(0)
cv2.destroyAllWindows()


ValueError: ignored